In [803]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [804]:
os.listdir(u'/Users/camilogarciatorres/Documents/GitHub/Taller1')

['.DS_Store',
 '.git',
 '.ipynb_checkpoints',
 'data',
 'output',
 'Taller1.ipynb',
 'Untitled.ipynb']

In [805]:
pwd


'/Users/camilogarciatorres/Documents/GitHub/Taller1'

In [806]:
base_dir = os.path.join( '.','data')
ciudad = 'Medellin'
fuente = 'NREL'
union = os.path.join(base_dir,ciudad,fuente+'.csv')
serieREL= pd.read_csv(union, sep = ';',
                encoding='latin-1')

base_dir2 = os.path.join( '.','data')
ciudad2 = 'Medellin'
fuente2 = 'IDEAM'
union2 = os.path.join(base_dir,ciudad,fuente+'.csv')
serieIDEAM = pd.read_csv(union2, sep = ';',
                encoding='latin-1')


In [807]:
def MCP(mediax, mediay, sigmax, sigmay, x):
    return ((mediay-(sigmay/sigmax)*mediax)+(sigmay/sigmax)*x)

In [808]:
serieIDEAM.head()

,AÑO,MES,DIA,HORA,NREL
0,2005,1,1,0,0.0
1,2005,1,1,1,0.0
2,2005,1,1,2,0.0
3,2005,1,1,3,0.0
4,2005,1,1,4,0.0


In [809]:
mediay=serieREL['NREL'].mean()
sigmay=serieREL['NREL'].std()

In [810]:
serieIDEAM.dtypes

AÑO       int64
MES       int64
DIA       int64
HORA      int64
NREL    float64
dtype: object

def pedazodedataframe(year1, year2, mes1, mes2, dia1, dia2,  serieIDEAM1):
    for year in range(year1,year2):
        for mes in range(mes1,mes2):
            for dia in range(dia1,dia2):
                listafinal= pd.DataFrame(columns=["AÑO", "MES", "DIAS", "GHIdia"])
                ghi_diario = serieIDEAM1.loc[(
                            (serieIDEAMl["AÑO"] == year)
                            & (serieIDEAM1["MES"] == mes))
                               "NREL"].item()
                ghi_dict = {"AÑO":agno, "MES":mes, "DIAs":dia,
                        "GHImes":ghi_diario}
                
                listafinal = listafinal.append(ghi_dict, ignore_index=True)
    return ghi_df



In [811]:
def pedazodedataframe(year1, year2, mes1, mes2, dia1, dia2, serieIDEAM1):
    
    pedazo= serieIDEAM1[(serieIDEAM1['AÑO']>=year1) & (serieIDEAM1['AÑO']<=year2)
                      & (serieIDEAM1['MES']>=mes1)& (serieIDEAM1['MES']<=mes2)
                      & (serieIDEAM1['DIA']>=dia1)& (serieIDEAM1['DIA']<=dia2)]

    return pedazo

In [812]:
z= serieIDEAM[(serieIDEAM['AÑO']>=2006) & (serieIDEAM['AÑO']<=2006)]

In [813]:
x=pedazodedataframe(2005, 2005, 1, 1, 1, 30,serieREL)
x

,AÑO,MES,DIA,HORA,NREL
0,2005,1,1,0,0.0
1,2005,1,1,1,0.0
2,2005,1,1,2,0.0
3,2005,1,1,3,0.0
4,2005,1,1,4,0.0
5,2005,1,1,5,0.0
6,2005,1,1,6,12.5
7,2005,1,1,7,103.5
8,2005,1,1,8,209.5
9,2005,1,1,9,478.0


In [814]:
def listaajustada(larga, corta):
    
    pedazolarga=larga[(larga['AÑO']>=corta.iloc[0,0]) & (larga['AÑO']<=corta.iloc[-1,0])]
    sigmax1=pedazolarga['NREL'].std()
    mediax1=pedazolarga['NREL'].mean()
    mediax2=corta['NREL'].mean()
    sigmax2=corta['NREL'].std()
    listafinal= pd.DataFrame(columns=["AÑO", "MES", "DIAS", "NREL"])
    correccion=MCP(mediax1, mediax2, sigmax1, sigmax2, x)
    listafinal = {"NREL":correccion}
    return listafinal
    
                                     #return listafinal

In [815]:
y=listaajustada(serieREL, z)
y

{'NREL':         AÑO  MES   DIA  HORA   NREL
 0    2005.0  1.0   1.0   0.0    0.0
 1    2005.0  1.0   1.0   1.0    0.0
 2    2005.0  1.0   1.0   2.0    0.0
 3    2005.0  1.0   1.0   3.0    0.0
 4    2005.0  1.0   1.0   4.0    0.0
 5    2005.0  1.0   1.0   5.0    0.0
 6    2005.0  1.0   1.0   6.0   12.5
 7    2005.0  1.0   1.0   7.0  103.5
 8    2005.0  1.0   1.0   8.0  209.5
 9    2005.0  1.0   1.0   9.0  478.0
 10   2005.0  1.0   1.0  10.0  806.0
 11   2005.0  1.0   1.0  11.0  787.5
 12   2005.0  1.0   1.0  12.0  821.0
 13   2005.0  1.0   1.0  13.0  792.5
 14   2005.0  1.0   1.0  14.0  679.0
 15   2005.0  1.0   1.0  15.0  573.5
 16   2005.0  1.0   1.0  16.0  347.0
 17   2005.0  1.0   1.0  17.0  121.0
 18   2005.0  1.0   1.0  18.0    0.0
 19   2005.0  1.0   1.0  19.0    0.0
 20   2005.0  1.0   1.0  20.0    0.0
 21   2005.0  1.0   1.0  21.0    0.0
 22   2005.0  1.0   1.0  22.0    0.0
 23   2005.0  1.0   1.0  23.0    0.0
 24   2005.0  1.0   2.0   0.0    0.0
 25   2005.0  1.0   2.0   1.0 

#PUNTO 4#
